動機
研界方法
如何處裡資料
實用性?

##Impoart module

In [ ]:
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt
from pandas.plotting import lag_plot
from pandas import datetime
from statsmodels.tsa.arima_model import ARIMA
from sklearn.metrics import mean_squared_error

In [ ]:
!pip install pmdarima

In [ ]:
import os
import warnings
warnings.filterwarnings('ignore')
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
plt.style.use('fivethirtyeight')
from pylab import rcParams
rcParams['figure.figsize'] = 10, 6
from statsmodels.tsa.stattools import adfuller
from statsmodels.tsa.seasonal import seasonal_decompose
from statsmodels.tsa.arima_model import ARIMA
from sklearn.metrics import mean_squared_error, mean_absolute_error
import math
import numpy as np
from pylab import rcParams


ImportError: ignored

In [ ]:
pip uninstall statsmodels

Uninstalling statsmodels-0.12.2:
  Would remove:
    /usr/local/lib/python3.7/dist-packages/statsmodels-0.12.2.dist-info/*
    /usr/local/lib/python3.7/dist-packages/statsmodels/*
Proceed (y/n)? y
  Successfully uninstalled statsmodels-0.12.2


In [ ]:
pip install statsmodels --upgrade

  Using cached https://files.pythonhosted.org/packages/da/69/8eef30a6237c54f3c0b524140e2975f4b1eea3489b45eb3339574fc8acee/statsmodels-0.12.2-cp37-cp37m-manylinux1_x86_64.whl


In [ ]:
pip install aws

     |████████████████████████████████| 1.4MB 5.8MB/s 
     |████████████████████████████████| 61kB 6.4MB/s 
     |████████████████████████████████| 215kB 47.8MB/s 
     |████████████████████████████████| 215kB 46.8MB/s 
     |████████████████████████████████| 962kB 36.8MB/s 
     |████████████████████████████████| 71kB 6.7MB/s 
     |████████████████████████████████| 3.2MB 33.9MB/s 
  Created wheel for aws: filename=aws-0.2.5-cp37-none-any.whl size=7464 sha256=5f822bd9f628e08b02f4d895a013d88b9719bc890400720cdc88506ff18b41bf
  Stored in directory: /root/.cache/pip/wheels/2a/c1/af/f03941c7a2b137f7c966c1d45d0661f0261e725929d0d12744
Successfully built aws


In [ ]:
pip install awscli

     |████████████████████████████████| 3.6MB 4.3MB/s 
     |████████████████████████████████| 7.5MB 33.7MB/s 
     |████████████████████████████████| 81kB 7.2MB/s 
     |████████████████████████████████| 552kB 35.5MB/s 
     |████████████████████████████████| 153kB 55.1MB/s 
ERROR: requests 2.23.0 has requirement urllib3!=1.25.0,!=1.25.1,<1.26,>=1.21.1, but you'll have urllib3 1.26.4 which is incompatible.
ERROR: datascience 0.10.6 has requirement folium==0.2.1, but you'll have folium 0.8.3 which is incompatible.
  Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3
  Found existing installation: docutils 0.17.1
    Uninstalling docutils-0.17.1:
      Successfully uninstalled docutils-0.17.1


In [ ]:
from pmdarima.arima import auto_arima


ImportError: ignored

#Data Prepocesess


##Tesla

In [ ]:
df = pd.read_csv("TSLA.csv")
df.head(5)

In [ ]:
df_close = df['Close']

In [ ]:
#see if there is some cross-correlation in out data.
plt.figure()
lag_plot(df['Open'], lag=3)
plt.title('TESLA Stock - Autocorrelation plot with lag = 3')
plt.show()

In [ ]:
#the stock price evolution over time.
plt.plot(df["Date"], df["Close"])
plt.xticks(np.arange(0,1259, 200), df['Date'][0:1259:200])
plt.title("TESLA stock price over time")
plt.xlabel("time")
plt.ylabel("price")
plt.show()

###ADF (Augmented Dickey-Fuller) Test

In [ ]:
#Test for staionarity
def test_stationarity(timeseries):
    #Determing rolling statistics
    rolmean = timeseries.rolling(12).mean()
    rolstd = timeseries.rolling(12).std()
    #Plot rolling statistics:
    plt.plot(timeseries, color='blue',label='Original')
    plt.plot(rolmean, color='red', label='Rolling Mean')
    plt.plot(rolstd, color='black', label = 'Rolling Std')
    plt.legend(loc='best')
    plt.title('Rolling Mean and Standard Deviation')
    plt.show(block=False)
    
    print("Results of dickey fuller test")
    adft = adfuller(timeseries,autolag='AIC')
    # output for dft will give us without defining what the values are.
    #hence we manually write what values does it explains using a for loop
    output = pd.Series(adft[0:4],index=['Test Statistics','p-value','No. of lags used','Number of observations used'])
    for key,values in adft[4].items():
        output['critical value (%s)'%key] =  values
    print(output)
    
test_stationarity(df_close)
#We see that the p-value is greater than 0.05 so we cannot reject the Null hypothesis. 
#Also, the test statistics is greater than the critical values. so the data is non-stationary.

In [ ]:
# separate Trend and Seasonality from the time series.
result = seasonal_decompose(df_close, model='multiplicative', freq = 30)
fig = plt.figure()  
fig = result.plot()  
fig.set_size_inches(16, 9)

In [ ]:
#we start by taking a log of the series to reduce the magnitude of the values and reduce the rising trend in the series. 
#Then after getting the log of the series, we find the rolling average of the series. 
#A rolling average is calculated by taking input for the past 12 months and giving a mean consumption value at every point further ahead in series.
rcParams['figure.figsize'] = 10, 6
df_log = np.log(df_close)
moving_avg = df_log.rolling(12).mean()
std_dev = df_log.rolling(12).std()
plt.legend(loc='best')
plt.title('Moving Average')
plt.plot(std_dev, color ="black", label = "Standard Deviation")
plt.plot(moving_avg, color="red", label = "Mean")
plt.legend()
plt.show()

In [ ]:
len(df_close)

In [ ]:
rcParams['figure.figsize'] = 10, 6
df_log_dif = np.log(df_close).diff()
moving_avg = df_log_dif.rolling(12).mean()
std_dev = df_log_dif.rolling(12).std()
plt.legend(loc='best')
plt.title('Moving Average')
plt.plot(std_dev, color ="black", label = "Standard Deviation")
plt.plot(moving_avg, color="red", label = "Mean")
plt.legend()
plt.show()

#Predictive by ARIMA model
**AR: < Auto Regressive >** means that the model uses the dependent relationship between an observation and some predefined number of lagged observations (also known as “time lag” or “lag”). \
**I:< Integrated >** means that the model employs differencing of raw observations (e.g. it subtracts an observation from an observation at the previous time step) in order to make the time-series stationary. \
**MA: < Moving Average >** means that the model exploits the relationship between the residual error and the observations.\
ref:\
https://towardsdatascience.com/time-series-forecasting-predicting-stock-prices-using-an-arima-model-2e3b3080bd70 \
https://www.kdnuggets.com/2020/01/stock-market-forecasting-time-series-analysis.html \
Time-series forecasting is widely used for non-stationary data.? \
\
 **Model parameters\**
The standard ARIMA models expect as input parameters 3 arguments i.e. p,d,q.\
p is the number of lag observations.\
d is the degree of differencing.\
q is the size/width of the moving average window.\

In [ ]:
#split data into train and training set
train_data, test_data = df_log[3:int(len(df_log)*0.79)], df_log[int(len(df_log)*0.79):]
plt.figure(figsize=(10,6))
plt.grid(True)
plt.xlabel('Dates')
plt.ylabel('Closing Prices')
plt.plot(df_log, 'green', label='Train data')
plt.plot(test_data, 'blue', label='Test data')
plt.legend()

In [ ]:
#split data into train and training set
train_data_d, test_data_d = df_log_dif[3:int(len(df_log_dif)*0.79)], df_log_dif[int(len(df_log_dif)*0.79):]
plt.figure(figsize=(10,6))
plt.grid(True)
plt.xlabel('Dates')
plt.ylabel('log difference of close price')
plt.plot(train_data_d, 'green', label='Train data')
plt.plot(test_data_d, 'blue', label='Test data')
plt.legend()

In [ ]:
model_autoARIMA = auto_arima(train_data, start_p=1, start_q=1,
                      test='adf',       # use adftest to find optimal 'd'
                      max_p=50, max_q=50, # maximum p and q
                      m=1,              # frequency of series
                      d=None,           # let model determine 'd'
                      seasonal=False,   # No Seasonality
                      start_P=0, 
                      D=0, 
                      trace=True,
                      error_action='ignore',  
                      suppress_warnings=True, 
                      stepwise=True)
print(model_autoARIMA.summary())


In [ ]:
model_autoARIMA_d = auto_arima(train_data_d, start_p=1, start_q=1,
                      test='adf',       # use adftest to find             optimal 'd'
                      max_p=10, max_q=10, # maximum p and q
                      m=1,              # frequency of series
                      d=None,           # let model determine 'd'
                      seasonal=False,   # No Seasonality
                      start_P=0, 
                      D=0, 
                      trace=True,
                      error_action='ignore',  
                      suppress_warnings=True, 
                      stepwise=True)
print(model_autoARIMA.summary())

In [ ]:
model_autoARIMA.plot_diagnostics(figsize=(15,8))
plt.show()

In [ ]:
model_autoARIMA_d.plot_diagnostics(figsize=(15,8))
plt.show()

Top left: The residual errors seem to fluctuate around a mean of zero and have a uniform variance.

Top Right: The density plot suggest normal distribution with mean zero.

Bottom left: All the dots should fall perfectly in line with the red line. Any significant deviations would imply the distribution is skewed.

Bottom Right: The Correlogram, aka, ACF plot shows the residual errors are not autocorrelated. Any autocorrelation would imply that there is some pattern in the residual errors which are not explained in the model. So you will need to look for more X’s (predictors) to the model.

In [ ]:
model = ARIMA(train_data, order=(0, 1, 0))  
fitted = model.fit(disp=-1)  
print(fitted.summary())

In [ ]:
fitted.plot_predict(dynamic=False)
plt.show()

In [ ]:
# Forecast
fc, se, conf = fitted.forecast(265, alpha=0.05)  # 95% confidence
fc_series = pd.Series(fc, index=test_data.index)
lower_series = pd.Series(conf[:, 0], index=test_data.index)
upper_series = pd.Series(conf[:, 1], index=test_data.index)
plt.figure(figsize=(12,5), dpi=100)
plt.plot(train_data, label='training')
plt.plot(test_data, color = 'blue', label='Actual Stock Price')
plt.plot(fc_series, color = 'orange',label='Predicted Stock Price')
plt.fill_between(lower_series.index, lower_series, upper_series, 
                 color='k', alpha=.10)
plt.title('Tesla Inc. Stock Price Prediction')
plt.xlabel('Time')
plt.ylabel('Actual Stock Price')
plt.legend(loc='upper left', fontsize=8)
plt.show()

In [ ]:
model_d = ARIMA(train_data_d, order=(0, 1, 0))  
fitted = model_d.fit(disp=-1)  
print(fitted.summary())

In [ ]:
fc, se, conf = fitted.forecast(265, alpha=0.95)  
fc_series = pd.Series(fc, index=test_data.index)
lower_series = pd.Series(conf[:, 0], index=test_data_d.index)
upper_series = pd.Series(conf[:, 1], index=test_data_d.index)
plt.figure(figsize=(12,5), dpi=100)
plt.plot(train_data_d, label='training')
plt.plot(test_data_d, color = 'blue', label='Actual Stock Price')
plt.plot(fc_series, color = 'orange',label='Predicted Stock Price')
plt.fill_between(lower_series.index, lower_series, upper_series, 
                 color='k', alpha=.10)
plt.title('Altaba Inc. Stock Price Prediction')
plt.xlabel('Time')
plt.ylabel('Actual Stock Price')
plt.legend(loc='upper left', fontsize=8)
plt.show()

In [ ]:
# report performance
mse = mean_squared_error(test_data, fc)
print('MSE: '+str(mse))
mae = mean_absolute_error(test_data, fc)
print('MAE: '+str(mae))
rmse = math.sqrt(mean_squared_error(test_data, fc))
print('RMSE: '+str(rmse))
mape = np.mean(np.abs(fc - test_data)/np.abs(test_data))
print('MAPE: '+str(mape))

# Predict by LSTM model

In [ ]:
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt
from pandas.plotting import lag_plot
from pandas import datetime
from statsmodels.tsa.arima_model import ARIMA
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import MinMaxScaler
import keras
from sklearn import metrics

In [ ]:
df = pd.read_csv("TSLA.csv")
print('Number of rows and columns:', df.shape)
df.head(5)

In [ ]:
df_log = np.log(df['Close'])
train_data, test_data = df_log[:1000], df_log[1000:]
plt.figure(figsize=(10,6))
plt.grid(True)
plt.xlabel('Dates')
plt.ylabel('Closing Prices')
plt.plot(df_log, 'green', label='training_set')
plt.plot(test_data, 'blue', label='test_set')
plt.legend()

In [ ]:
training_set = np.log(df.iloc[:1000, 1:2].values)
test_set = np.log(df.iloc[1000:, 1:2].values)
t = MinMaxScaler(feature_range = (0, 1))
training_set_scaled = t.fit_transform(training_set)
X_train = []
Y_train = []
for i in range(60, 1000):
    X_train.append(training_set_scaled[i-60:i, 0])
    Y_train.append(training_set_scaled[i, 0])
X_train, Y_train = np.array(X_train), np.array(Y_train)
X_train = np.reshape(X_train, (X_train.shape[0], X_train.shape[1], 1))

In [ ]:
model = keras.models.Sequential()

model.add(keras.layers.LSTM(units = 150, return_sequences = True, input_shape = (X_train.shape[1], 1)))
model.add(keras.layers.Dropout(0.25))

model.add(keras.layers.LSTM(units = 150, return_sequences = True))
model.add(keras.layers.Dropout(0.25))

model.add(keras.layers.LSTM(units = 150, return_sequences = True))
model.add(keras.layers.Dropout(0.25))

model.add(keras.layers.LSTM(units = 150))
model.add(keras.layers.Dropout(0.25))

model.add(keras.layers.Dense(units = 1))

model.compile(optimizer = 'adam', loss = 'mean_squared_error')

model.fit(X_train, Y_train, epochs = 300, batch_size = 100)

In [ ]:
dataset_train = np.log(df.iloc[:1000, 1:2])
dataset_test = np.log(df.iloc[1000:, 1:2])
dataset_total = pd.concat((dataset_train, dataset_test), axis = 0)
inputs = dataset_total[len(dataset_total) - len(dataset_test) - 60:].values
inputs = inputs.reshape(-1,1)
inputs = t.transform(inputs)
X_test = []
for i in range(60, 319):
    X_test.append(inputs[i-60:i, 0])
X_test = np.array(X_test)
X_test = np.reshape(X_test, (X_test.shape[0], X_test.shape[1], 1))
print(X_test.shape)

In [ ]:
predicted_stock_price = model.predict(X_test)
predicted_stock_price = t.inverse_transform(predicted_stock_price)

In [ ]:
plt.figure(figsize=(15,6))
plt.plot(df.loc[1000:, 'Date'],dataset_test.values, color = 'red', label = 'Real TESLA Stock Price')
plt.plot(df.loc[1000:, 'Date'],predicted_stock_price, color = 'blue', label = 'Predicted TESLA Stock Price')
plt.xticks(np.arange(0,259,50))
plt.title('TESLA Stock Price Prediction')
plt.xlabel('Time')
plt.ylabel('TESLA Stock Price')
plt.legend()
plt.show()

In [ ]:
Real = df.loc[1000:, 'Date'],dataset_test.values
Predicted = df.loc[1000:, 'Date'],predicted_stock_price
y_real = np.array(Real[1])
y_pred = np.array(Predicted[1])
def mape(y_real, y_pred):
    return np.mean(np.abs((y_pred - y_real) / y_real)) * 100

def smape(y_real, y_pred):
    return 2.0 * np.mean(np.abs(y_pred - y_real) / (np.abs(y_pred) + np.abs(y_real))) * 100

# MSE
MSE = metrics.mean_squared_error(y_real, y_pred)
print('MSE: '+str(MSE)) 
# RMSE
RMSE = np.sqrt(metrics.mean_squared_error(y_real, y_pred))
print('RSME: '+str(RMSE)) 
# MAE
MAE = metrics.mean_absolute_error(y_real, y_pred)
print('MAE: '+str(MAE)) 
# MAPE
MAPE = mape(y_real, y_pred)
print('MAPE:'+str(MAPE))